In [1]:
%pip install torchdata==0.10.1
%pip install --upgrade transformers
%pip install --upgrade sentence-transformers

Note: you may need to restart the kernel to use updated packages.
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.2
    Uninstalling transformers-4.27.2:
      Successfully uninstalled transformers-4.27.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

%pip install  wandb


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Ignored the following yanked versions: 0.3.0a0
ERROR: Could not find a version that satisfies the requirement torchdata==0.5.1 (from versions: 0.3.0a1, 0.3.0, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.8.0, 0.9.0, 0.10.0, 0.10.1, 0.11.0)
ERROR: No matching distribution found for torchdata==0.5.1
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.27.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!nvidia-smi

Wed Feb 26 12:02:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  |   00000000:01:00.0 Off |                  N/A |
| 35%   38C    P8             26W /  250W |       1MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
PROJECT = "Dialogue-Summarizer"
MODEL_NAME = 'google/flan-t5-base'
DATASET = "knkarthick/dialogsum"

In [6]:
import wandb
wandb.init(project=PROJECT,
           tags=[MODEL_NAME, DATASET],
           notes ="Fine tuning FlanT5 with PEFT: LoRA")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: benahmeed-myriam (benahmeed-myriam-university-of-passau) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch
import time
import evaluate
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

2025-02-26 12:02:54.242370: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-26 12:02:54.242403: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 12:02:54.242420: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 12:02:54.247142: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-26 12:02:54.909046: W tensorflow/compiler/

# **I. Load Dataset and Flan-T5**

In [8]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

In [9]:
with wandb.init(project=PROJECT, job_type="dataset"):
   wbtrain = wandb.Table(data=dataset['train'].to_pandas())
   wbvalidation = wandb.Table(data=dataset['validation'].to_pandas())
   wbtest = wandb.Table(data=dataset['test'].to_pandas())
   wandb.log({"dialogsum_train": wbtrain})
   wandb.log({"dialogsum_validation": wbvalidation})
   wandb.log({"dialogsum_test": wbtest})


In [10]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exa

In [11]:
#Load the model : Flan-T5: creating an instance of the AutoModelForSeq2SeqLM class with the .from_pretrained() method.
model_name='google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
#Download the tokenizer related to FLAN-T5: Tokenization is the process of splitting texts into smaller units that can be processed by the LLM models.
#Download the tokenizer for the FLAN-T5 model using AutoTokenizer.from_pretrained() method. Parameter use_fast switches on fast tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [13]:
#test the Tokenizer
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

DECODED SENTENCE:
What time is it, Tom?


In [14]:
# determine the nbr of model parameters and how many of them are trainable
def print_number_of_trainable_model_parameters(model, tag="original_model"):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    with wandb.init(project=PROJECT, job_type="log_parameters"):
      wandb.log({f'{tag}': {"trainable_model_params":trainable_model_params}})
      wandb.log({f'{tag}': {"all_model_params":all_model_params}})
      wandb.log({f'{tag}': {"percentage_of_trainable_model_parameters": 100 * trainable_model_params}} )

    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.0%


In [15]:
#explore how well the base LLM summarizes a dialogue : this an inference and evaluation script
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
       original_model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

-------------------------------

In [16]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

In [17]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

In [18]:
tokenized_datasets['train'].to_pandas().head()

,input_ids,labels
0,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[1363, 5, 3931, 31, 7, 652, 3, 9, 691, 18, 413..."
1,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[1713, 345, 13515, 536, 4663, 65, 3, 9, 682, 2..."
2,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[1713, 345, 13515, 357, 4663, 987, 7, 1713, 34..."
3,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[1713, 345, 13515, 536, 4663, 4277, 7, 3, 9, 1..."
4,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[1713, 345, 13515, 536, 4663, 11, 1713, 345, 1..."


In [19]:
with wandb.init(project=PROJECT, job_type="dataset"):
   wbtrain_tokenized = wandb.Table(data=tokenized_datasets['train'].to_pandas())
   wbvalidation_tokenized = wandb.Table(data=tokenized_datasets['validation'].to_pandas())
   wbtest_tokenized = wandb.Table(data=tokenized_datasets['test'].to_pandas())
   wandb.log({"dialogsum_train_tokenized": wbtrain_tokenized})
   wandb.log({"dialogsum_validation_tokenized": wbvalidation_tokenized})
   wandb.log({"dialogsum_test_tokenized": wbtest_tokenized})

In [20]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


# **II. Perform Parameter Efficient Fine-Tuning (PEFT: LoRA)**

In [43]:
from types import SimpleNamespace
from pathlib import Path
from tqdm.notebook import tqdm
from datetime import datetime
from peft import LoraConfig, get_peft_model, TaskType

In [44]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

config = SimpleNamespace(
    # hyperparameters
    learning_rate=1e-3,
    gradient_accumulation_steps=2,
    num_train_epochs=20,
    save_steps=1000,
    save_strategy='steps', # we cannot set it to "no". Otherwise, the model cannot guess the best checkpoint.
    eval_steps=1000,
    logging_steps=1000,
    evaluation_strategy="steps",
    warmup_steps=500,
    save_total_limit=3,
    load_best_model_at_end = True,
    output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}',
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.03,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM, # FLAN-T5
    auto_find_batch_size=True,
)

In [45]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=config.r,
    lora_alpha=config.lora_alpha,
    target_modules=config.target_modules,
    lora_dropout=config.lora_dropout,
    bias=config.bias,
    task_type=config.task_type # FLAN-T5
)

In [46]:
#Add LoRA adapter layers/parameters to the original LLM to be trained.
peft_model = get_peft_model(original_model,lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.4092820552029972%


In [48]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=peft_model
)

In [49]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    auto_find_batch_size=config.auto_find_batch_size,
    output_dir=config.output_dir,
    learning_rate=config.learning_rate,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    num_train_epochs=config.num_train_epochs,
    save_steps=config.save_steps,
    save_strategy=config.save_strategy, # we cannot set it to "no". Otherwise, the model cannot guess the best checkpoint.
    eval_steps=config.eval_steps,
    logging_steps=config.logging_steps,
    evaluation_strategy=config.evaluation_strategy,
    warmup_steps=config.warmup_steps,
    save_total_limit=config.save_total_limit,
    load_best_model_at_end = config.load_best_model_at_end,
    report_to="wandb",
    run_name=f"PEFT_tuning-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"

)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator
)

In [50]:
#train
with wandb.init(project=PROJECT, job_type="train"):
      peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss,Validation Loss


train/epoch,▁
train/global_step,▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,20
train/global_step,320
train/total_flos,1739072471040000.0
train/train_loss,10.47517


NameError: name 'peft_model_path' is not defined

In [52]:
#Evaluation
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")


peft_model = PeftModel.from_pretrained(peft_model_base,
                                       peft_model_path,
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [53]:
peft_model = peft_model.to("cuda")

In [54]:
# Define W&B Table to store generations
columns = ["index", "dialoge", "prompt", "human_sumary","peft_model_output"]
table3 = wandb.Table(columns=columns)

In [55]:
lindex = [100,200,300]
for index in lindex:
  dialogue = dataset['test'][index]['dialogue']
  baseline_human_summary = dataset['test'][index]['summary']

  prompt = f"""
  Summarize the following conversation.

  {dialogue}

  Summary: """

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to("cuda")
    
  original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


  peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)


print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is crazy for Trump and voted for him. #Person2# doesn't agree with #Person1# on Trump and will vote for Biden.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1# is proud to say that he is our President again. #Person2# is not sure about this. #Person2# isn't sure about it. #Person1# will vote for Biden.
---------------------------------------------------------------------------------------------------
PEFT MODEL: #Person1# and #Person2# are proud to say that Trump is our President. #Person1# is proud to say that he is our President. #Person2# is not sure about this. #Person2# has nothing but faith in Trump. #Person2# is not sure about this. #Person2# will vote for Biden.


In [56]:
with wandb.init(project=PROJECT, job_type="examples"):

   wandb.log({"peft_model": table3})

In [57]:
rouge = evaluate.load('rouge')

In [58]:
# Evaluate the Model Quantitatively (with ROUGE Metric)
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to("cuda")


    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1# requests a memorandum from #Person1#...,#Person1# asks #Person2# to take a dictation f...
1,In order to prevent employees from wasting tim...,#Person1# and #Person1# are discussing a memo ...,#Person1# asks #Person2# to take a dictation f...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person2# needs to take a dictation for #Perso...,#Person1# asks #Person2# to take a dictation f...
3,#Person2# arrives late because of traffic jam....,#Person1# is busy with traffic. #Person2# thin...,#Person2# got stuck in traffic. #Person2# thin...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1# is stuck in traffic. #Person2# feels...,#Person2# got stuck in traffic. #Person2# thin...
5,#Person2# complains to #Person1# about the tra...,#Person2# wants to take public transport to wo...,#Person2# got stuck in traffic. #Person2# thin...
6,#Person1# tells Kate that Masha and Hero get d...,#Person1# and Masha and Hero are getting divor...,#Person1# and Masha are getting divorced. #Per...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced. #Person2#...,#Person1# and Masha are getting divorced. #Per...
8,#Person1# and Kate talk about the divorce betw...,#Person2# and Hero are getting divorced. #Pers...,#Person1# and Masha are getting divorced. #Per...
9,#Person1# and Brian are at the birthday party ...,#Person1# and #Person2# danced at the party. #...,#Person1# invites Brian to the party. #Person2...


In [60]:
with wandb.init(project=PROJECT, job_type="examples"):
  table2= wandb.Table(data=df)
  wandb.log({"outputs_original_instruct_peft_model": table2})

In [61]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)


peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.28790549573455443, 'rouge2': 0.1116146484371235, 'rougeL': 0.24632345973809386, 'rougeLsum': 0.24798834184024326}
PEFT MODEL:
{'rouge1': 0.2396307589608096, 'rouge2': 0.05352384140832349, 'rougeL': 0.2046081743765003, 'rougeLsum': 0.20543517854870993}


In [62]:
human_baseline_summaries = df['human_baseline_summaries'].values
original_model_summaries = df['original_model_summaries'].values
peft_model_summaries     = df['peft_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.28790549573455443, 'rouge2': 0.1116146484371235, 'rougeL': 0.24632345973809386, 'rougeLsum': 0.24798834184024326}
PEFT MODEL:
{'rouge1': 0.2396307589608096, 'rouge2': 0.05352384140832349, 'rougeL': 0.2046081743765003, 'rougeLsum': 0.20543517854870993}


In [63]:
# Define W&B Table to store generations
columns = ["metric", "original_model","improvement"]
table10 = wandb.Table(columns=columns)

In [64]:
print("Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE")
with wandb.init(project=PROJECT, job_type="metrics"):
  #peft model
  improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
  for key, value, inst, ori in zip(peft_model_results.keys(), improvement, peft_model_results.values(),original_model_results.values() ):
      print(f'{key}: {value*100:.2f}% original = {ori} instruct = {inst}' )
      table10.add_data(key, ori, f"{value*100:.2f}%")

  wandb.log({"Rouge Metrics": table10})

Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE


rouge1: -4.83% original = 0.28790549573455443 instruct = 0.2396307589608096
rouge2: -5.81% original = 0.1116146484371235 instruct = 0.05352384140832349
rougeL: -4.17% original = 0.24632345973809386 instruct = 0.2046081743765003
rougeLsum: -4.26% original = 0.24798834184024326 instruct = 0.20543517854870993


In [66]:
results_path = "./results/results_flat_t5.csv"
df.to_csv(results_path, index=False)

In [ ]:
wandb.finish()